In [36]:
import urllib.request as request
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

In [37]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [38]:
request.urlretrieve("https://github.com/chatgpt-kr/openai-api-tutorial/raw/main/ch07/2020_%EA%B2%BD%EC%A0%9C%EA%B8%88%EC%9C%B5%EC%9A%A9%EC%96%B4%20700%EC%84%A0_%EA%B2%8C%EC%8B%9C.pdf",
                    "data-files/economy-dictionary.pdf")

('data-files/economy-dictionary.pdf',
 <http.client.HTTPMessage at 0x1c74119c580>)

In [39]:
%%time
# pdf 파일의 텍스트 정보를 읽어서 페이지 단위로 반환
loader = PyPDFLoader('data-files/economy-dictionary.pdf')
pages = loader.load_and_split()

CPU times: total: 15.8 s
Wall time: 16 s


In [40]:
print( len(pages) )
# print( pages[0].page_content )
# print( pages[12].page_content ) # 0 ~ 12번 문서는 목차
# print( pages[-1].page_content ) # 마지막 문서는 기타 정보

pages2 = pages[13:-1] # 불필요한 문서(페이지) 제거
print(len(pages2))

366
352


In [41]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

In [42]:
documents = text_splitter.split_documents(pages2)

In [43]:
len(documents)

646

In [44]:
# 분할된 문서들을 vectordb에 저장 (여기서는 Chroma)

chroma_db = Chroma.from_documents(documents, OpenAIEmbeddings()) # 메모리에 저장
# chroma_db = Chroma.from_documents(documents, OpenAIEmbeddings(), 
#                                   persist_directory="vectordb/chroma.economy.db") # 파일로 저장

In [45]:
chroma_db._collection.count()

1292

In [46]:
# 데이터 저장 구조 (구성) 확인
# for k, v in chroma_db._collection.get().items(): # items() : key, value from dict
#     print(k)

for k in chroma_db._collection.get(): # key from dict
    print(k)

ids
embeddings
documents
uris
data
metadatas
included


In [47]:
chroma_db._collection.get()["ids"] # 데이터 반환
chroma_db._collection.get()["embeddings"] # 어떤 데이터는 None으로 반환
chroma_db._collection.get(include=["embeddings"])['embeddings']

array([[-0.0077088 , -0.0075166 ,  0.014182  , ..., -0.01861645,
        -0.00669286, -0.00903364],
       [ 0.00170697, -0.01484938,  0.03096656, ...,  0.00016448,
        -0.00243612, -0.01603626],
       [-0.01100211, -0.00490143,  0.03516901, ..., -0.00774796,
         0.00382345, -0.02089929],
       ...,
       [-0.00025882, -0.01862838,  0.05028324, ..., -0.01868199,
        -0.00552486, -0.00399371],
       [-0.01951408, -0.03514976,  0.0151068 , ..., -0.00689231,
        -0.00983841, -0.01012996],
       [-0.00895678, -0.02865633,  0.02298952, ...,  0.00560302,
        -0.00409904, -0.00425682]])

In [48]:
embeddings = chroma_db._collection.get(include=["embeddings"])['embeddings']

In [49]:
embeddings.shape

(1292, 1536)

In [50]:
# 유사도가 높은 2개의 문서를 반환하는 반환기로 설정
retriever = chroma_db.as_retriever(search_kwargs={"k":2}) 

In [51]:
docs = retriever.get_relevant_documents("비트코인을 알려주세요")

In [52]:
print(len(docs))
print(docs[0])

2
page_content='에 게재된다. 단기지표금리로서 코리보의 육성 노력에도 불구하고 금리호가의 기초가 
되는 은행간 무담보 기일물거래가 미미하여 신뢰성이 낮아 일부 은행에서 변동금리대출, 
기간물 콜거래 준거금리 및 내부이전금리로 제한적으로 활용되는 데 그치고 있다. 장기
적으로는 은행간 단기 자금거래뿐만 아니라 코리보 연동대출, 변동금리채권(FRN) 발행, 
이자율스왑 등 다양한 금융거래에서 시장참여자들의 가격결정 기준으로 활용되고 예금･
대출시장, 채권시장, 파생금융상품시장 등 여러 금융시장 간의 연계성을 높여 금융산업 
선진화에도 일정부분 기여할 것으로 기대되고 있다 .
 연관검색어 : LIBOR, 자금조달비용지수(COFIX)
코리보 ∙' metadata={'page': 309, 'source': 'data-files/economy-dictionary.pdf'}


In [53]:
# 템플릿 만들기
template = """
당신은 한국은행에서 만든 금융 용어를 설명해주는 도우미입니다.
주어진 검색 결과를 기반으로 답변해 주세요.
검색 결과에 없는 내용은 답변할 수 없다고 해주세요.

[검색결과]
{context}

질문: {question}
답변:"""

prompt = PromptTemplate.from_template(template=template)

In [54]:
llm = ChatOpenAI(model="gpt-4o", temperature=0)

In [55]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type_kwargs={"prompt": prompt},
    retriever=retriever,
    return_source_documents=True
)

In [56]:
response = qa_chain.invoke("비트코인은 무엇인가요?")
response

{'query': '비트코인은 무엇인가요?',
 'result': '비트코인(bitcoin)은 가상통화(암호통화)이자 디지털 지급시스템입니다. 비트코인 시스템은 중앙 저장소 또는 단일 관리자가 없기 때문에 최초의 탈중앙화된 디지털통화라고 불립니다. 이는 사토시 나카모토라는 사람(또는 집단)에 의해 만들어져 2009년 개방형 소프트웨어로 배포되었습니다. 비트코인 시스템은 공유형(peer-to-peer)으로, 거래는 중개자 없이 블록체인 소프트웨어를 이용하는 참여자(nodes) 사이에 직접 이루어집니다. 거래는 참여자의 작업증명(proof-of-work)을 통해 검증되고, 공개된 분산원장인 블록체인에 기록됩니다. 비트코인은 채굴(mining)에 대한 보상으로 발행되며, 다른 통화, 상품, 용역 등과 교환될 수 있습니다. 비트코인은 중앙은행이 발행한 법정화폐가 아니며, 투자대상으로도 관심을 받고 있습니다. 그러나 가격의 급등 및 변동 폭으로 인해 거품 논란이 있으며, 익명성으로 인해 자금세탁 등 불법거래에 악용될 소지가 있습니다. 비트코인의 총량은 2100만 개로 한정되어 있으며, 2140년경 모두 채굴될 것으로 전망됩니다.',
 'source_documents': [Document(metadata={'page': 155, 'source': 'data-files/economy-dictionary.pdf'}, page_content='139\nㅂ \n비트코인\n비트코인(bitcoin)은 가상통화(암호통화)이자 디지털 지급시스템이다. 비트코인 시스템\n은 중앙 저장소 또는 단일 관리자가 없기 때문에 최초의 탈중앙화된 디지털통화라고 불린다. \n이는 사토시 나카모토라는 사람(집단)에 의해 만들어져서 2009년 개방형 소프트웨어로 \n배포되었다. 이 시스템은 공유형(peer-to-peer)이며, 거래는 중개자 없이 블록체인 소프트\n웨어를 이용하는 참여자(nodes) 사이에 직접 이뤄진다. 이런 거래들은 공유(P2P) 네트워크\n상 참여자의 작업증명(proof-of-wor

In [57]:
response = qa_chain.invoke("너 누구야?")
response

{'query': '너 누구야?',
 'result': '죄송하지만, 주어진 검색 결과에 해당하는 정보가 없습니다. 다른 질문을 해주시면 감사하겠습니다.',
 'source_documents': [Document(metadata={'page': 204, 'source': 'data-files/economy-dictionary.pdf'}, page_content='대상가구는 소득기준(국민기초생활보장법에 따른 생계급여 또는 의료급여 수급자) 에 \n따른 수급자 본인이나 가구원이 가구원특성기준(노인, 영유아, 장애인 또는 임산부)을 \n충족할 경우에만 수혜를 받을 수 있다 . \n 연관검색어 : 사회보장제도, 사회보험\n엥겔의 법칙\n엥겔의 법칙은 독일의 통계학자 엥겔이 벨기에 노동자의 가계조사를 통해 발견하였다.'),
  Document(metadata={'page': 204, 'source': 'data-files/economy-dictionary.pdf'}, page_content='대상가구는 소득기준(국민기초생활보장법에 따른 생계급여 또는 의료급여 수급자) 에 \n따른 수급자 본인이나 가구원이 가구원특성기준(노인, 영유아, 장애인 또는 임산부)을 \n충족할 경우에만 수혜를 받을 수 있다 . \n 연관검색어 : 사회보장제도, 사회보험\n엥겔의 법칙\n엥겔의 법칙은 독일의 통계학자 엥겔이 벨기에 노동자의 가계조사를 통해 발견하였다.')]}

In [58]:
# !pip install gradio==4.44.1

In [59]:
def get_chatbot_response(message):
    chatbot_response = qa_chain.invoke(message)
    return chatbot_response["result"].strip()

In [60]:
import gradio as gr

# 인터페이스를 생성.
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="경제금융용어 챗봇") # 경제금융용어 챗봇 레이블을 좌측 상단에 구성
    msg = gr.Textbox(label="질문해주세요!")  # 하단의 채팅창의 레이블
    clear = gr.Button("대화 초기화")  # 대화 초기화 버튼

    # 챗봇의 답변을 처리하는 함수
    def respond(message, chat_history):
      bot_message = get_chatbot_response(message)

      # 채팅 기록에 사용자의 메시지와 봇의 응답을 추가.
      chat_history.append((message, bot_message))
      return "", chat_history

    # 사용자의 입력을 제출(submit)하면 respond 함수가 호출.
    msg.submit(respond, [msg, chatbot], [msg, chatbot])

    # '초기화' 버튼을 클릭하면 채팅 기록을 초기화.
    clear.click(lambda: None, None, chatbot, queue=False)

# 인터페이스 실행.
demo.launch(debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.
